Testing of ResNet+MLP where we see how well it performs when trained using a NeRF but tested on 'actual' camera data.

In [ ]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

In [ ]:
import data_functions as df
import training_functions as tf
from models import *

In [ ]:
# Initialize NN variables
hidden_sizes = [
    [256, 128, 7],
    [128, 32, 7],
    [128, 32, 8, 7]
    ]

mlps = [VisionPoseMLP(hidden_size,True) for hidden_size in hidden_sizes]

Neps = 300

In [ ]:
# Generate data loaders
train_loader, test_loader = df.get_data(0.8)

In [ ]:
# Train the basic models
for idx,mlp in enumerate(mlps):
    mlp_name = "basic"+str(idx+1).zfill(3)
    print("=============================================================")
    print("Training: "+mlp_name)
    tf.train_model(mlp, train_loader, mlp_name, Neps)
    print("-------------------------------------------------------------")
    tf.test_model(mlp, test_loader)